In [1]:
import ray 
import gym 
import numpy as np 

In [12]:
class TestEnv(gym.Env):
    def __init__(self, env_config):
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Discrete(100)
        
    def reset(self):
        self.state = np.random.randint(100)        
        return self.state
    
    def step(self, action):
        self.state += 1 if action ==1 else -1
        if self.state <=0 or self.state >=99:
            done =True 
        else:
            done=False 
            
        return self.state, -100, done, {}
    
class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env, config):
        super().__init__(env)
        
    def reset(self):
        obs = self.env.reset()
        self.timestep = 0
        return obs 
    
    def step(self, action):
        state, reward, done, info = self.env.step(action)
        
        if self.timestep >100:
            done = True
        if done:
            reward = 100 - self.timestep
        else:
            reward = 0
        return state, reward, done, info


def deeping_env_creator(env_config):
    env = TestEnv(env_config)
    env = RewardWrapper(env, env_config)
    return env 

from ray.tune import register_env

register_env("test_env", deeping_env_creator)


In [13]:
analysis = ray.tune.run('DQN', config={"env":"test_env", "framework":"torch"}, stop={"training_iteration":2} )

== Status ==
Current time: 2022-02-10 14:40:10 (running for 00:00:00.11)
Memory usage on this node: 14.4/16.0 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/8.44 GiB heap, 0.0/4.22 GiB objects
Result logdir: /Users/bumjin/ray_results/DQN
Number of trials: 1/1 (1 PENDING)
+--------------------------+----------+-------+
| Trial name               | status   | loc   |
|--------------------------+----------+-------|
| DQN_test_env_e9055_00000 | PENDING  |       |
+--------------------------+----------+-------+




(DQNTrainer pid=6613) 2022-02-10 14:40:11,176	INFO simple_q.py:153 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQNTrainer pid=6613) 2022-02-10 14:40:11,177	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQNTrainer pid=6613) 2022-02-10 14:40:11,198	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(DQNTrainer pid=6613) 2022-02-10 14:40:11,199	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Result for DQN_test_env_e9055_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2022-02-10_14-40-12
  done: false
  episode_len_mean: 997.0
  episode_media: {}
  episode_reward_max: 100.0
  episode_reward_mean: 100.0
  episode_reward_min: 100.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 797275ba0b6b4e2cb6999971431c8815
  hostname: bagbeomjins-MacBook-Pro.local
  info:
    last_target_update_ts: 1000
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_lr: 0.0005
          grad_gnorm: 0.10335072875022888
          max_q: 0.07963812351226807
          mean_q: -0.1031285896897316
          min_q: -0.4210588335990906
        mean_td_error: -0.01788289099931717
        model: {}
        td_error:
        - 0.11982420086860657
        - -0.004023246467113495
        - -0.12588927149772644
        - -0.054803140461444855
        - -0.017689034342765808
        - 0.1207609623670578


2022-02-10 14:40:15,530	INFO tune.py:636 -- Total run time: 5.63 seconds (5.35 seconds for the tuning loop).
